# <span style = "color:black">**Introduction**

variational auto-encoder는 **generative model**로서 intractable한 posterior를 포함하는 latent variable이 있고 large dataset에도 잘 동작한다. **이는 1)variational lower bound를 reparamet-<br>erization을 통해 샘플링**하고 SGD를 사용하며 **2)posterior를 variational inference로 구한 뒤 encoder(inference model)로 학습**시키기에 가능하다. 이번 포스트에서는 논문에서 사용된 방법들에 대해서 자세하게 수식적으로 설명한다.

# <span style = "color:black">**Background**

## <span style = "color:black">**Latent Variable Model**

latent variable model은 **관측가능한 데이터를 변수와 관련짓는 통계학적 모델**이다. 이때의 **변수를 latent variable이라고 하며 데이터가 어떻게 생성되는지에 영향**을 미친다. latent variable model에서 데이터가 만들어지는 과정은 다음과 같다.

**(Data Generation Process)**<br>
**1.** latent variable $\bf{z}$가 $p(\bf{z})$에서 먼저 sampling된다. <br>
**2.** observed data $\bf{x}$가 그 후  $p({\bf{x}}|{\bf{z}}) = p({\bf{x}}|f_{\theta}({\bf{z}}))$에서 sampling된다. <br>

- $\bf{z}$의 dimension은 일반적으로 $\bf{x}$의 dimension보다 작다고 가정한다.
- 여기서 $f_\theta(\bf{z})$는 $\theta$를 parameter로 갖는 vector function으로 여러개(또는 하나의)의 output을 가집니다.

예를 들어 학교에 "고양이"를 주제로 하여 과제를 제출해야 한다고 가정해보자. latent variable은 교수님이 지정한 글자크기,글자간격,글자수,주제(고양이)로 비유할 수 있으며 제출할 과제는 이러한 잠재변수들이 결과를 미쳤을 것이라 볼 수 있다. 특히 주목할 점은 latent variable $\bf{z}$가 그대로 영향을 미칠 수도 있고 적절한 transform(function)을 거친 $f_{\theta}(\bf{z})$가 영향을 미칠 수도 있다는 것이다. 이는 주제가 "고양이"로 선정되었어도 누군가는 "고양이에게 줘야하는 음식"으로 다른 누군가는 "고양이의 행동분석"으로 과제를 제출하는 것에 비유할 수 있다.

## <span style = "color:black">**Generative Model**

Generative Model(생성모델)은 observed data와 유사하면서도 다른 new samples을 생성해내는 것이 목적이다. 이를 다시말하면 **Generative model의 목적은 $p_{data}(x)$를 구하는 것**이라고 할 수 있다. 관측된 데이터를 생성하는 확률분포 $p_{data}(\bf{x})$를 알면 sampling을 통해서 생성해낼 수 있기 때문이다.

# <span style = "color:black">**Method**

$p_{data}(\bf{x})$는 어떻게 구할 수 있을까? VAE에서는 Maximum likelyhood estimation으로 확률분포를 구하고자 한다. 구체적인 단계는 다음과 같다.

(**Probability density estimation**)<br>
**1.** $p_{data}(\bf{x})$를 근사할 likelyhood $p(\bf{x})$를 가정<br>
**2.** likelyhood가 어떤 확률분포를 따를지 결정(ex 정규분포,베르누이분포 ...)<br>
**3.** likelyhood를 가장 크게하는 parameter를 $p(\bf{x})$의 parameter로 설정 <br>
**4.** $p({\bf{x}}) \approx p_{data}({\bf{x}}) \rightarrow p(\bf{x})$를 $p_{data}(\bf{x})$대신 사용!!

1. 먼저 $p(\bf{x})$를 구해보자. Generation process가 latent variable model이라고 하면 다음과 같은 explicit form으로 적어줄 수 있다.

$$p({\bf{x}}) = \int_z p({\bf{x}}|{\bf{z}})p({\bf{z}})d{\bf{z}} = \int_z p({\bf{x}}|{f_{\theta}(\bf{z})})p({\bf{z}})d{\bf{z}}$$

2. $p(\bf{x})$를 결정하는 것은 적분안의 함수를 모두 결정하는 것과 같다. VAE에서 $p({\bf{x}}|f_{\theta}({\bf{z}}))$는 $\bf{x}$가 실수전체에서 연속적일 경우 정규분포로 0과1사이에서 연속적일 경우 베르누이분포로 가정한다. $p({\bf{x}}|f_{\theta}{(\bf{z})}) = \mathcal{N}({\bf{x}}|f_{\theta}(\bf{z}),\sigma^2 * \bf{I})$로 가정하고 전개해보자. 

\begin{aligned}
p({\bf{x}}) &= \int_z p({\bf{x}}|{f_{\theta}(\bf{z})})p({\bf{z}})d{\bf{z}}\\
&= \int_z \mathcal{N}({\bf{x}}|f_{\theta}({\bf{z}}),\sigma^2 * {\bf{I}})p({\bf{z}})d{\bf{z}}
\end{aligned}

- $f_{\theta}(z)$가 정규분포의 parameter(평균)이므로 명시적으로 $\boldsymbol{\theta}$표시. $ p({\bf{x}}) \rightarrow p(\bf{x}|\boldsymbol{\theta})$
- $f_{\theta}(z)$는 사실 Deep Nueral Network이자 매우 복잡한 함수임

여기서 latent variable의 distribution인 $p(\bf{z})$만 잘 결정해주면 우리는 monte carlo method를 사용하여 likelyhood를 근사하고 Gradient Ascent를 사용하면 될 것이다. 그러나 이는 쉽지않은 조심스러운 문제이다.

예를 들어서 숫자이미지를 생성하는 latent variable을 생각해보자. 단순하게 생각하면 어떤 숫자를 적어야 하는지에 관한 변수가 있지만 이 외에도 굵기,각도,스타일등의 **여러가지 우리가 알지못하는 변수들이 데이터의 생성에 영향**을 미칠 수 있다. 또한 각도가 휘어있는 숫자는 누군가가 빨리 쓰다가 그랬다고 생각할 수 있으며 이는 상당히 얇은 굵기를 만들어 낼 수도 있으므로 **변수들간에 상관관계도 존재**할 수 있다. 요약하자면 **latent variable의 distribution은 매우 복잡하다는 것**이다.

그렇다면 VAE에서는 어떻게 $p(\bf{z})$를 가정할까? 그냥 **단순히 평균이 0이며 상관관계가 존재하지 않는(공분산이 존재하지 않는) 가장 단순한 정규분포로 가정**해버린다.

$${\bf{z}} \sim p({\bf{z}}) = \mathcal{N}({\bf{z}}|0,\bf{I})$$

당연하게도 이렇게 **단순하게 분포를 가정해버린다면 복잡한 latent variable의 분포를 전혀 대표하지 못할 것 같다**라는 생각이 든다. 왜냐하면 **너무 쉬운 latent variable의 distribution을**가정했기 때문이다. 여기서 **복잡한 함수로서 역할을 하는 Deep Neural Network**가 이 문제를 해결해줄 수 있다. 조금 더 세부적으로 확인해보자.

\begin{aligned}
p({\bf{x}}) &= \int_z \mathcal{N}({\bf{x}}|f_{\theta}({\bf{z}}),\sigma^2 * {\bf{I}})p({\bf{z}})d{\bf{z}}
\end{aligned}

수식으로 위의 문제를 다시보면 $f_{\theta}({\bf{z}})$가 만약 극단적으로 Identity mapping이라면 정규분포에서 $z$가 sampling되는데 그대로 값이 들어온다면 실제 Data는 복잡한 확률분포로부터 sampling한 latent variable의 영향을 받는데 이를 전혀 반영하지 못한다는 것이다. 그러므로 vae에서는 $f_{\theta}({\bf{z}})$를 Deep Nueral Network를 사용한다.

![probabilistic decoder - Matthew N.Bernstein's github](./images/decoder.PNG)

위의 그림에서  $\bf{z}$는 DNN인 $f_{\theta}({\bf{z}})$의 input이 된다. 여기서 중요한 두가지는 첫째로 **DNN인 $f_{\theta}({\bf{z}})$뿐만 아니라 처음 몇개의 히든레이어도 복잡한 함수의 역할을 할 수 있다**는 것이고 두번째는 **변수가 임의의 함수를 통과하면 달라진 분포를 갖는다**라는 것이다.

![Tutorial on Variational Autoencoders - figure2](./images/complicated%20mapping.PNG)

- ${\bf{z}} \sim \mathcal{N}(z|0,\bf{I})$
- $g({\bf{z}}) = \bf{z/10 + z/||z||}$

예를 들어 위의 그림을 보자. 왼쪽은 단순히 $\bf{z}$가 Gaussian distribution을 따르므로 sample을 확인해보면 근사적으로 Gaussian 분포를 이룬다. 오른쪽 그림은 DNN,Hidden layer와는 다르게 그리 복잡한 함수는 아닌 단순한 $g({\bf{z}})$에 대한 함숫값들의 분포이다. 비교적 간단한 mapping임에도 함숫값의 분포는 ring 형태를 이루는 것으로 보아 $g(\bf{z})$는 $\bf{z}$보다는 좀 더 복잡한 분포를 갖는다는 것을 알 수 있다. DNN에서는 이보다 더 복잡하다. 여러개의 **hidden layer 에 대한 output은 $\bf{z}$에 대한 훨씬 더 복잡한 함수의 함숫값**이므로 단순한 ring 형태뿐만 아니라 **더 복잡한 분포를 따르는 확률변수**라고 생각할 수 있다.

이렇게 DNN에서 존재하는 **hiddenlayer의 output은 더 복잡한 분포를 가지는 확률변수이므로 새로운 latent variable의 역할을 한다.** 새로운 latent variable로부터 나온 값이 이제 활용되어 나머지 hidden layer를 통과하며 새로운 데이터(이미지 ..)를 만들어낼 수 있다. 한 가지 신기한 점은 분포의 모양은 DNN의 학습을 통해 결정된다는 사실이다. training set에 있는 데이터와 비슷한 sample을 만들어내기 위해서 적절한 함수를 학습하여 적절한 분포를 가진다.

다음과 같은 data set 주어져 있다고 해보자.

\begin{aligned}
&{\bf{X}} = \{{\bf{x}^{(i)}}\}_{n=1}^N\\
&\,\text{consisiting of N i.i.d samples of discrete(or) random variable {\bf{x}}} \in \mathbb{R}^K\\
\end{aligned}

Data set은 다음과 같은 random process에 의해서 생성된다고 가정한다.

1. ${\bf{z}}^{i} \in \mathbb{R}^P$는 $p_{{\boldsymbol{\theta}}^*}({\bf{z}})$에서 먼저 생성되었다. (generated) (단, $K>P$)
2. ${\bf{x^{(i)}}}$가 ${\bf{z^{(i)}}}$를 condition으로 하는 distribution $p_{{\boldsymbol{\theta}}^*}({\bf{x}}|{\bf{z}} = {\bf{z^{(i)}}})$에서 생성되었다.

또한 계산의 간단함을 위해서 prior $p_{{\boldsymbol{\theta}}^*}(\bf{z})$와 likelyhood $p_{{\boldsymbol{\theta}}^*}\bf{(x|z)}$는 동일한 파라미터를 가지는 분포 $p_{{\boldsymbol{\theta}}}(\bf{z})$와 $p_{{\boldsymbol{\theta}}}(\bf{x|z})$이며 ${\boldsymbol{\theta}},z$에 대하여 거의 모든 곳에서 미분가능하다고 가정한다.

- ${\bf{z}^{(i)}}$는 i-번째 생성된 latent variable이다. 이 randomvariable은 우리는 알려지지 않았다.
- ${\bf{x}^{(i)}}$는 i-번째 latent variable을 condition으로 하는 conditional distribution으로부터 sampling된 값(벡터)이다.
- $p_{{\boldsymbol{\theta}}}$는 parameter가 ${\boldsymbol{\theta}}$인 확률분포함수이다. 예를들어 정규분포라면 ${\boldsymbol{\theta}} = \{\boldsymbol{\mu},\boldsymbol{\sigma^2}\}$이다.

생성모델을 만들고자 할때 나타나는 일반적인 상황은 다음과 같다.

- 일반적으로 생성모델은 $p_{\boldsymbol{\theta}}{\bf{({\bf{x}})}} = \int_z p_{\boldsymbol{\theta}}({\bf{z}})p_{\boldsymbol{\theta}}({\bf{x|z}})d{\bf{z}}$가 $\bf{z}$의 차원이 너무커서 적분을 할 수 없음.  따라서 posterior를 구할 수 없는 Intractability가 존재.(Note : likelyhood와 prior를 쉬운 분포로 가정해놔도 계산할 수 없음!)
- large dataset이 주어져서 batch단위로 optimization을 하기에는 너무 큰 computational cost가 예상됨.

논문에서 위와 같은 문제점을 극복하여 다음의 것들을 문제를 풀고자 한다.

1. True Parameter인 ${\boldsymbol{\theta}^*}$를 추정하여 random process 모방. 즉, True $p_{{\boldsymbol{\theta}}}{\bf{(x|z)}}$를 학습하여 latent variable에서 input space로의 mapping 학습. ($\boldsymbol{\theta} \rightarrow \boldsymbol{\theta}^*$)
2. True Posterior $p_{{\boldsymbol{\theta}^*}}{\bf{({\bf{z|x}})}}$를 구하여 representation,coding task에서의 사용.($\boldsymbol{\theta}^*$를 알고있어도 구하는 것이 불가능했던 확률분포)


결국에 보면 encooder,decoder를 학습하는 문제이다.

# <span style = "color:black">**Method**

## <span style = "color:black">**posterior inference via Variational Inference**

2번째 문제인 True posterior를 구하기 위해서는 어떻게 해야할까? 먼저 $\boldsymbol{\theta} \rightarrow \boldsymbol{\theta}^*$가 되도록 학습(1번째 목적)해야하며 또한 True parameter $\boldsymbol{\theta^*}$를 찾았다고 해도 intractability를 해결하여 Posterior를 구해야 한다. True parameter를 찾는 방법은 일단 재껴두고 여기서는 **임의의 $\boldsymbol{\theta}$으로 parameterize된 posterior $p_{\boldsymbol{\theta}}(\bf{{\bf{z|x}}})$를 구하는 것에만 집중**한다. 또한 먼저 논의를 간단히 하기 위해서 single observation ${\bf{x}}$에 대해서만 고려한다.

Posterior $p_{\boldsymbol{\theta}}(\bf{{\bf{z|x}}})$는 어떻게 구할 수 있을까? 논문에서는 **variational inference의 방법** 즉, **$\bf{z}$에 관한 또다른 확률분포 $q_{\boldsymbol{\phi}}\bf{({\bf{z|x}})}$를 가정한 후 이를 점점 실제 posterior인 $p_{{\boldsymbol{\theta}}}\bf{({\bf{z|x}})}$에 다가가도록 하는 것**이다. **$q_{{\boldsymbol{\phi}}}$가 충분히 $p_{\boldsymbol{\theta}}$에 다가가면 이를 posterior 대신 사용**한다. 단,여기서 $q_{\boldsymbol{\phi}}$는 계속해서 ${\boldsymbol{\phi}}$를 수정해 나가야하므로 우리가 알고있는 계산이 편리한 함수로 가정한다.(예를 들면 정규분포)

위와 같은 방법을 사용하기 위해서는 두 분포사이의 "차이"를 좁혀나가야 하며 이를 알려주는 값인 KL-divergence를 사용한다.

\begin{aligned}
\text{D}_{KL}[q_{\boldsymbol{\phi}}({\bf{z|x}})||p_{{\boldsymbol{\theta}}}({\bf{z|x}})] &:= \int_zq_{\boldsymbol{\phi}}({\bf{z|x}})\text{log}\frac{q_{\boldsymbol{\phi}}({\bf{z|x}})}{p_{{\boldsymbol{\theta}}}({\bf{z|x}})}dz \\
\end{aligned}

그렇다면 2번째 목적은 다음과 같이 KL-divergence를 가장 줄이는 $\hat{q_{\boldsymbol{\phi}}}$를 찾는 것과 같다.

$${\hat{q_{\boldsymbol{\phi}}}} = \underset{q_{\boldsymbol{\phi}}}{\text{argmin}}\,\text{D}_{KL}[q_{\boldsymbol{\phi}}({\bf{z|x}})||p_{{\boldsymbol{\theta}}}({\bf{z|x}})] $$

여기서 RHS term인 KL-divergence는 ELBO와 evidence(marginal log likelyhood)의 차이로 전개할 수 있다.

\begin{aligned}
&
\begin{aligned}
\text{D}_{KL}[q_{\boldsymbol{\phi}}({\bf{z|x}})||p_{{\boldsymbol{\theta}}}({\bf{z|x}})] &:= \int_zq_{\boldsymbol{\phi}|}({\bf{z|x}})\text{log}\frac{q_{\boldsymbol{\phi}}({\bf{z|x}})}{p_{{\boldsymbol{\theta}}}({\bf{z|x}})}dz \\
&= -\mathcal{L}(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{x}}) + \text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x}})\\
\end{aligned}
\\
&\text{where } \mathcal{L}(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{x}}) := \mathbb{E}_{Z\sim q}\left[\text{log}\,\frac{p_{{\boldsymbol{\theta}}}({\bf{x,z}})}{q_{\boldsymbol{\phi}}({\bf{z|x}})} \right] = \mathbb{E}_{Z\sim q}\left[\text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x,z}})-\text{log}\,q_{\boldsymbol{\phi}}({\bf{z|x}}) \right]
\end{aligned}



- $\mathcal{L}(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{x}})$는 evidence = $\text{log}p_{\boldsymbol{\theta}}(\bf{x})$의 lowerbound(하한)이다. i.e. $\text{log}\,p_{\boldsymbol{\theta}}(\bf{x}) \geq \mathcal{L}(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{x}})$ 줄여서 ELBO라고 한다. 
- ELBO를 $\boldsymbol{\phi},{\boldsymbol{\theta}}$의 함수로 notation한 이유는 optimization과정에서 parameter를 변수이기.(MLE에서 parameter는 하나의 고정된 상수이지만 일단은 변수로 보고 likelyhood의 max값을 찾는 과정과 비슷한 느낌인 것 같다.)
- ${\bf{x}}$는 single observation으로 고정된 관측값이다. 우리의 목적은 관측된 값 ${\bf{x}}$가 있을 때의 latent variable ${\bf{z}}$의 posterior를 구하는 것임을 기억하자.

여기서 $p_{{\boldsymbol{\theta}}}({\bf{x}})$는 $q_{\boldsymbol{\phi}}$와 독립적이므로 $q_{\boldsymbol{\phi}}$를 maximization하는 과정에서는 상수취급 할 수 있으므로 이는 ELBO를 maximization하는 문제로 바뀌게 된다.

\begin{aligned}
\hat{q_{\boldsymbol{\phi}}} &= \underset{q_{\boldsymbol{\phi}}}{\text{argmax}}\,\mathcal{L}(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{x}}) \\
&= \underset{q_{\boldsymbol{\phi}}}{\text{argmax}}\,\mathbb{E}_{Z\sim q}\left[\text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x,z}})-\text{log}\,q_{\boldsymbol{\phi}}({\bf{z|x}}) \right]
\end{aligned}

또한 ${\bf{z}}$는 integral후에 사라지는 적분변수이며 ${\bf{x}}$는 관측값이다. 결국 $q_{\boldsymbol{\phi}}$를 control하여 KL-divergence를 maximize하는 것은 $\boldsymbol{\phi}$를 control하여 KL-divergence를 maximize하는 $\hat{\boldsymbol{\phi}}$를 찾는 것과 같은 문제이다.

\begin{aligned}
\hat{\boldsymbol{\phi}} = \underset{\boldsymbol{\phi}}{\text{argmax}}\,\mathcal{L}(\boldsymbol{\phi};{\boldsymbol{\theta}},x)
\end{aligned}

- maximization 과정에서는 $\boldsymbol{\phi}$만 움직이는 변수 취급하므로 ${\boldsymbol{\theta}}$는 일단 고정된 값으로 취급하였다.

정리하자면 문제는 원래 우리가 임의로 설정한 posterior인 $q_{\boldsymbol{\phi}}$를 posterior인 $p_{\boldsymbol{\theta}}$와 비슷하게 만드는 문제였다. 이는 KL-divergence를 minimize하는 $q_{\boldsymbol{\phi}}$를 찾는 것이었으며 수식전개를 통해서 결국에는 ELBO를 maximize하는 $\boldsymbol{\phi}$를 찾는 문제와 같음을 알 수 있다. 문제를 끝까지 푸는 건 잠시 뒤로 미루고 1번째 문제를 푸는 방법에 대해서 생각해보자.

## <span style = "color:black">**estimate $\boldsymbol{\theta}^*$ via MLE**

위에서 2번 문제를 푸는 방법을 알았기에 이제 해결해야 할 것은 1번문제인 true parameter인 $\theta^*$를 구하는 것이다. 이 문제도 풀게되면 True posterior,True likelyhood,True prior을 근사적으로 구하며 randomprocess(encoder,decoder)를 묘사할 수 있다.

우리는 true parameter인 $\theta^*$에 가장 근접하도록 $\theta$를 구해야 하기 때문에 먼저 $\theta^*$에 대해 생각해보자. $\theta^*$은 과연 어떤 값일까? 아마도 $\theta^*$는 관측된 데이터 ${\bf{x}}$에 대해서 가장 높은 확률을 부여하는 값일 것이다. 따라서 우리는 관측된 데이터에 대해서 가장 높은 확률을 부여하는 $\theta$를 찾아야 한다. 이는 전체적으로 MLE의 방법과 동일하다.

\begin{aligned}
\hat{\theta} = \underset{{\bf{\theta}}}{\text{argmax}}\,\text{log}\,p_{\boldsymbol{\theta}}(\bf{x})
\end{aligned}

문제는 $p_{\boldsymbol{\theta}}{\bf{({\bf{x}})}} = \int_z p_{\boldsymbol{\theta}}({\bf{z}})p_{\boldsymbol{\theta}}({\bf{x|z}})d{\bf{z}}$이다. problem setting에서 언급했듯이 구할 수 없는 값이다. 여기서 문제를 약간 우회하여 $\text{log}\,p_{\boldsymbol{\theta}}(\bf{x})$의 lowerbound인 ELBO를 maximize하는 문제로 바꾼다. 즉 다음과 같다.

\begin{aligned}
&\hat{\theta} = \underset{{\bf{\theta}}}{\text{argmax}}\,\text{log}\,p_{\boldsymbol{\theta}}(\bf{x}) \approx \underset{{\bf{\theta}}}{\text{argmax}}\,\mathcal{L}({\boldsymbol{\theta}};\boldsymbol{\phi},{\bf{x}}) \\
&\text{where }\\
&\text{evidence} := \text{log}p_{\boldsymbol{\theta}}({\bf{x}})\\
&\text{log}\,p_{\boldsymbol{\theta}}({\bf{x}}) \geq \text{ELBO} := \mathcal{L}(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{x}})=\mathbb{E}_{Z\sim q}\left[\text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x,z}})-\text{log}\,q_{\boldsymbol{\phi}}({\bf{z|x}}) \right] 
\end{aligned}

- $\theta$만 바꿔가며 위의 문제를 풀게되므로 $\boldsymbol{\phi}$는 fixed된 값이며 ;뒤에 위치시켰다.

물론 ELBO도 $z$에 대한 적분이기 때문에 풀 수 없음을 알 수 있다. 하지만 이제 문제가 ELBO를 optimization문제로 바뀌었음을 알 수 있으며 우리는 ELBO 하나의 값에 대해서 집중적으로 풀어낼 수 있는 방법을 사용할 것이다.(쉽게말하면 ELBO 하나만 샘플링해서 풀기위해서 이렇게 바꾼 것이다.)

\begin{aligned}
&\boldsymbol{\theta,\phi}=\underset{{\boldsymbol{\theta,\phi}}}{\text{argmax}}\,\mathcal{L}(\boldsymbol{\phi},{\boldsymbol{\theta}};\bf{x})\\
&\text{where }\mathcal{L}(\boldsymbol{\phi},{\boldsymbol{\theta}},{\bf{x}}) = \mathbb{E}_{Z\sim q}\left[\text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x,z}})-\text{log}\,q_{\boldsymbol{\phi}}({\bf{z|x}}) \right] 
\end{aligned}

- $\boldsymbol{\theta,\phi}$ : parameters of function $q$ and $p$
- ${\bf{x}}$ : single observation

## <span style = "color:black">**Solving Optimization Problem**

문제는 전적으로 이제 ELBO를 Optimization하는 문제로 바뀌었다. 우리는 ELBO에 Gradient ascent를 사용하여 optimal한 $\hat{\boldsymbol{\phi}},\hat{\boldsymbol{\theta}}$를 구할것이다. 그러기전에 먼저 ELBO를 살펴보자.

\begin{aligned}
\mathcal{L}(\boldsymbol{\phi},{\boldsymbol{\theta}},{\bf{x}}) &= \mathbb{E}_{Z\sim q}\left[\text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x,z}})-\text{log}\,q_{\boldsymbol{\phi}}({\bf{z|x}}) \right] \\
&= \int_z\left[\text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x,z}})-\text{log}\,q_{\boldsymbol{\phi}}({\bf{z|x}})\right]q({\bf{z|x}})d{\bf{z}}
\end{aligned}


이전에도 계속 문제였던 $d\bf{{z}}$가 또 등장한다. 그러므로 우리는 expectation을 구할 수 없다. 따라서 samping을 통해 expectation을 근사적으로 구하는 Monte carlo method를 사용한다. 이는 다음과 같다.

\begin{aligned}
\mathcal{L}(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{x}}) &= \mathbb{E}_{Z\sim q}\left[\text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x,z}})-\text{log}\,q_{\boldsymbol{\phi}}({\bf{z|x}}) \right] \\
&= \int_z\left[\text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x,z}})-\text{log}\,q_{\boldsymbol{\phi}}({\bf{z|x}})\right]q({\bf{z|x}})d{\bf{z}} \\
&\approx \frac{1}{L}\sum_{l=1}^L[\text{log}\,p_{\boldsymbol{\theta}}({\bf{x,z}}^{(l)}) - \text{log}\,q_{\boldsymbol{{\phi}}}({\bf{z}}|{\bf{x}}^{(l)})]
\end{aligned}
$$\text{where } {\bf{z}}^{(l)} \sim q_{\boldsymbol{\phi}}({\bf{z}})$$


ELBO를 근사적으로 구했으므로 최적화를 위해 gradient를 구해야 한다. 먼저 $\theta$에 대한 gradient를 구해보자.

\begin{aligned}
\nabla_{\theta}\mathcal{L}(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{x}}) &= \mathbb{E}_{Z\sim q}\left[\nabla_\theta\text{log}\,p_{\boldsymbol{\theta}}({\bf{x,z}})\right]\\
&\approx \frac{1}{L}\sum_{l=1}^L\nabla_\theta\text{log}\,p_{\boldsymbol{\theta}}({\bf{x,z}}^{(l)})
\end{aligned}

- 뒷항은 $\theta$에 대해 parameterize되어있지 않으므로 미분과정에서 제거됨.

$\theta$에 대한 gradient는 전혀 문제없이 잘 구해짐을 알 수 있다. 하지만 문제는 $\phi$에 대한 gradient를 구할 때 발생한다. $f({\bf{z}}) = \text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x,z}})-\text{log}\,q_{\boldsymbol{\phi}}({\bf{z|x}})$라 할 때 gradient는 다음과 같이 구해진다.(증명생략)

\begin{aligned}
\nabla_{\phi}\mathbb{E}_{q_{\boldsymbol{\phi}}({\bf{z})}}[f({\bf{z}})] &= \mathbb{E}_{q_{\boldsymbol{\phi}(\bf{z})}} \nabla_{q_{\boldsymbol{\phi}(\bf{z})}}\text{log}\,q_{\phi}({\bf{z}}) \\&\approx \frac{1}{L}\sum_{l=1}^L f({\bf{z}})\nabla_{q_{\boldsymbol{\phi}({\bf{z}})}}\text{log}\,q_{\boldsymbol{\phi}}({\bf{z}}^{(l)})
\end{aligned}

갑자기 로그가 들어간 형태로 gradient가 구해진다. 이러한 gradient에 대한 추정량은 unbiased estimator이지만 high variance를 가진다고 한다. 그러므로 샘플링을 아주많이(거의무한하게)취하는 것이 아니면 이 값은 수렴하지 않는 값이므로 적당한 샘플링을 통해서 문제를 풀어야 하는 우리의 방식에는 맞지않는다. 그러므로 샘플을 대체적으로 취하는 방식인 reparameterization을 사용한다. 이를 사용하면 unbiased estimator를 얻을뿐만 아니라 low variance를 가진다. (더 자세한 논의는 [링크](https://stats.stackexchange.com/questions/199605/how-does-the-reparameterization-trick-for-vaes-work-and-why-is-it-important)에서 확인 가능)

**(reparameterization trick)**<br>
${\bf{z}}$가 conditional distribution인 $q_{\phi}(\bf{z|x})$를 따르는 continuous 또는 discrete random-variable일때 ${\bf{z}} = g_{\phi}(\boldsymbol{\epsilon},\bf{x})$인 g에 대해 deterministic한 random variable로 나타낼 수 있다는 것이다.(where, ${\bf{\epsilon}}\sim p(\boldsymbol{\epsilon})$)

그러므로 우리는 reparameterization trick을 사용해서 ELBO를 Monte carlo method를 통해 다르게 생각할 수 있다. 이렇게 ELBO를 구하면 Gradient estimator는 unbiased일 뿐만 아니라 low variance를 가진다.

\begin{aligned}
\tilde{\mathcal{L}}^A(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{x}}) &= \mathbb{E}_{Z\sim q}\left[\text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x,z}})-\text{log}\,q_{\boldsymbol{\phi}}({\bf{z|x}}) \right] \\
&= \int_z\left[\text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x,z}})-\text{log}\,q_{\boldsymbol{\phi}}({\bf{z|x}})\right]q({\bf{z|x}})d{\bf{z}} \\
&\approx \frac{1}{L}\sum_{l=1}^L[\text{log}\,p_{\boldsymbol{\theta}}({\bf{x,z}}^{(l)}) - \text{log}\,q_{\boldsymbol{{\phi}}}({\bf{z}}|{\bf{x}}^{(l)})]
\end{aligned}
$$\text{where } {\bf{z}}^{(l)} =  q_{\boldsymbol{\phi}}({\boldsymbol{\epsilon}}^{(l)},{\bf{x}}),{\boldsymbol{\epsilon}}\sim p(\boldsymbol{\epsilon})

- A는 type A를 의미함.

논문에는 이러한 estimator말고도 다른 방법으로 구한 것도 있다. 이는 다음과 같다. 

\begin{aligned}
\tilde{\mathcal{L}}^B(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{x}}) = -D_{KL}(q_{\boldsymbol{\phi}}({\bf{z|x}})||p_\theta({\bf{z}})) + \frac{1}{L}\sum_{l=1}^L(\text{log}\,p_\theta({\bf{x}}|{\bf{z}}^{(l)})
\end{aligned}
$$\text{where } {\bf{z}}^{(l)} =  q_{\boldsymbol{\phi}}({\boldsymbol{\epsilon}}^{(l)},{\bf{x}}),{\boldsymbol{\epsilon}}\sim p(\boldsymbol{\epsilon})

두번째 estimator를 통해서 ELBO를 maximization하는 최적화 문제를 푸는것이 왜 auto-encoder와 연결되는지 알 수 있다. ELBO를 maximize하려면 첫번째 term을 가능한 작게 해야 하는데 이는 observation을 보고 latent variable가 따르는 예측한 값이 얼마나 차이가 나느냐이다.(encoder의 성능?인 것 같다.)두번째 텀은 decoder가 얼마나 latenr variable에서 input space로 mapping을 잘 하느냐를 의미한다.(decoder의 성능?)

여기까지 단 한개의 관측치에 대해서 모든 과정을 수행해봤다. 사실은 한 개의 관측치가 아니라 data set의 크기가 m인 mini-batch에 대하여 위의 과정을 수행해줘야 하므로 이것을 고려한 ELBO는 다음과 같다.

\begin{aligned}
\mathcal{L}(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{X}}) \approx 
\tilde{L}^M(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{X}})=\frac{N}{M}\sum_{i=1}^M\tilde{L}(\boldsymbol{\theta},\boldsymbol{\phi};{\bf{x}}^{(i)})
\end{aligned}
$$\text{where } {\bf{z}}^{(l)} \sim q_{\boldsymbol{\phi}}({\bf{z}})$$


# <span style = "color:black">**Appendix**

# <span style = "color:black">**evidence lower bound(ELBO)**

먼저 evidence는 다음과 같이 marginzalization된 loglikelyhood로 정의한다.

$$\text{evidence} := \text{log}\,p(x;{\boldsymbol{\theta}}) = \int_z p_{{\boldsymbol{\theta}}}({\bf{x,z}})dz$$

evidence를 쭉 전개하면 다음과 같다.

\begin{aligned}
\text{log}\,p(x;{\boldsymbol{\theta}}) &= \int_z p_{{\boldsymbol{\theta}}}({\bf{x,z}})dz \\
&=\text{log}\,\int_z \frac{p_{{\boldsymbol{\theta}}}({\bf{x,z}})}{q_{\boldsymbol{\phi}}({\bf{z|x}})}q_{\boldsymbol{\phi}}({\bf{z|x}})dz \\
&=\text{log}\,\mathbb{E}_{Z\sim q}\left[\frac{p_{{\boldsymbol{\theta}}}({\bf{x,z}})}{q_{\boldsymbol{\phi}}({\bf{z|x}})} \right]
\end{aligned}

[Jenson's inequality](https://ko.wikipedia.org/wiki/%EC%98%8C%EC%84%BC_%EB%B6%80%EB%93%B1%EC%8B%9D) $f(E[X])\leq E[f({\bf{x}})]$에 의하여 evidence의 lowerbound를 찾을 수 있다. 이를 evidence lower bound(ELBO)라 정의한다.

\begin{aligned}
&\text{log}\,p(x;{\boldsymbol{\theta}}) \geq \mathbb{E}_{Z\sim q}\left[\text{log}\,\frac{p_{{\boldsymbol{\theta}}}({\bf{x,z}})}{q_{\boldsymbol{\phi}}({\bf{z|x}})} \right]\\
&\mathcal{L(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{x}})} := \mathbb{E}_{Z\sim q}\left[\text{log}\,\frac{p_{{\boldsymbol{\theta}}}({\bf{x,z}})}{q_{\boldsymbol{\phi}}({\bf{z|x}})} \right] = \mathbb{E}_{Z\sim q}\left[\text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x,z}})-\text{log}\,q_{\boldsymbol{\phi}}({\bf{z|x}}) \right]
\end{aligned}

- evidence를 $\boldsymbol{\phi},{\boldsymbol{\theta}}$의 함수로 notation한 이유는 optimization 과정에서 변수로 보고 사용하기 때문이다.(MLE에서 parameter는 하나의 고정된 상수이지만 일단은 변수로 보고 likelyhood의 max값을 찾는 과정과 비슷한 느낌인 것 같다.)
- 관측값${\bf{x}}$는 고정이므로 ;뒤에 놓았다.(이것도 MLE에서 관측한 값은 고정인 느낌과 같다.)

# <span style = "color:black">**KL-divergence 전개**

\begin{aligned}
\text{D}_{KL}[q_{\boldsymbol{\phi}}({\bf{z|x}})||p_{\boldsymbol{\theta}}({\bf{z|x}})] &:= \int_zq_{\boldsymbol{\phi}}
({\bf{z|x}})\text{log}\frac{q_{\boldsymbol{\phi}}({\bf{z|x}})}{p_{{\boldsymbol{\theta}}}({\bf{z|x}})}dz \\
&= \int_zq_{\boldsymbol{\phi}}({\bf{z|x}})\text{log}\frac{q_{\boldsymbol{\phi}}({\bf{z|x}})p_{{\boldsymbol{\theta}}}({\bf{x}})}{p_{\boldsymbol{\theta}}({\bf{x,z}})}dz \\
&= \int_zq_{\boldsymbol{\phi}}({\bf{z|x}})\text{log}\frac{q_{\boldsymbol{\phi}}({\bf{z|x}})}{p_{{\boldsymbol{\theta}}}({\bf{x,z}})}dz  + \int_z q_{\boldsymbol{\phi}}({\bf{z|x}})\text{log}\,p_{\boldsymbol{\theta}}({\bf{x}})dz\\
&= -\mathcal{L}(\boldsymbol{\phi},{\boldsymbol{\theta}};{\bf{x}}) + \text{log}\,p_{{\boldsymbol{\theta}}}({\bf{x}})
\end{aligned}